In [ ]:
import pandas as pd
import plotly.express as px

import pandas as pd
from datetime import datetime
import numpyro
from numpyro import distributions as dist
from numpyro import infer
from jax import jit, random
import arviz as az
from estival.sampling import tools as esamp
from plotly.express.colors import qualitative as qual_colours
from IPython.display import Markdown

from emu_renewal.outputs import get_spaghetti_from_params, get_quant_df_from_spaghetti, plot_spaghetti, plot_uncertainty_patches
from emu_renewal.process import CosineMultiCurve
from emu_renewal.distributions import GammaDens
from emu_renewal.renew import RenewalModel
from emu_renewal.calibration import StandardCalib

In [ ]:
# Grab data on 2014-2016 Ebola outbreak - this analysis we will focus on the epidemic in Seirra Leone. This data was don

ebola_data = pd.read_csv(r'C:\Users\ehug0006\emu\wpro-working\data\ebola_2014_2016_clean.csv', index_col="Date", parse_dates=True)

In [ ]:
fig = px.line(ebola_data, x=ebola_data.index, y='Cumulative no. of confirmed, probable and suspected cases', color='Country')

fig.show()

In [ ]:
ebola_data

In [ ]:
case_data = ebola_data[ebola_data['Country']=='Sierra Leone']['Cumulative no. of confirmed, probable and suspected cases'].diff().rolling(14).mean().dropna()

In [ ]:
case_data

In [ ]:
fig = px.line(case_data, x=case_data.index, y=case_data)

fig.show()

In [ ]:
# Specify fixed parameters and get calibration data
run_in = 10
proc_update_freq = 4
pop = 7.1e6
analysis_start = ebola_data.index[0]
analysis_end = datetime(2015, 5, 1)
select_data = case_data.loc[analysis_start: analysis_end]

In [ ]:
fitter = CosineMultiCurve()
renew_model = RenewalModel(33e6, analysis_start, analysis_end, run_in, proc_update_freq, fitter, GammaDens(), fitter, 50)

In [ ]:
calib = StandardCalib(renew_model, select_data)

In [ ]:
# Define parameter ranges
priors = {
    "gen_mean": dist.Uniform(10.0, 14.0),
    "gen_sd": dist.Uniform(3.0, 7.0),
    "cdr": dist.Beta(10.0, 4.0),
    "seed": dist.Uniform(0.4, 1.5),
}

In [ ]:
kernel = numpyro.infer.NUTS(calib.calibration, dense_mass=True, init_strategy=infer.init_to_uniform(radius=0.5))
mcmc = numpyro.infer.MCMC(kernel, num_chains=2, num_samples=1000, num_warmup=1000)
rng_key = random.PRNGKey(1)
mcmc.run(rng_key, extra_fields=("accept_prob",), params=priors)

In [ ]:
idata = az.from_numpyro(mcmc)

In [ ]:
burn_in = 10
n_samples = 100
quantiles = [0.05, 0.5, 0.95]
idata_burnt = idata.sel(draw=slice(burn_in, None))
idata_sampled = az.extract(idata_burnt, num_samples=n_samples)
sample_params = esamp.xarray_to_sampleiterator(idata_sampled)

In [ ]:
def get_full_result(gen_mean, gen_sd, proc, seed, cdr):
    return renew_model.renewal_func(gen_mean, gen_sd, proc, seed)

full_wrap = jit(get_full_result)
spaghetti = get_spaghetti_from_params(renew_model, sample_params, full_wrap)
quantiles_df = get_quant_df_from_spaghetti(renew_model, spaghetti, quantiles)

In [ ]:
plot_uncertainty_patches(quantiles_df, select_data, qual_colours.Plotly)